# install packages

In [1]:
!pip install torch==2.0.1 torchtext==0.15.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [1]:
!pip install datasets
!pip install spacy
!pip install transformers
!pip install hazm

# import packages

In [2]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from datasets import Dataset as D
import torch.nn.functional as F
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import random
from transformers import AutoTokenizer
import sentencepiece as spm
from torch.nn import Embedding
from gensim.models import Word2Vec
import numpy as np
from transformers import MBartTokenizer

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Neural machine Translation

## Download dataset

In [5]:
from datasets import load_dataset

# Load the Global Voices English-Persian translation dataset
dataset = load_dataset("persiannlp/parsinlu_translation_en_fa", split='train')

# Print a sample of the dataset
print("Sample English-Persian sentence pair:")
print(dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.39k [00:00<?, ?B/s]

parsinlu_translation_en_fa.py:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

parsinlu-repo/validation/0000.parquet:   0%|          | 0.00/242k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1621665 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/48359 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2137 [00:00<?, ? examples/s]

Sample English-Persian sentence pair:
{'source': 'Due Thank You note by Egyptian blogger Abdel Monem Mahmoud: Following his release from prison, he wrote his first blog titled “Ana ikhwan… I am free”.', 'targets': ['بلاگر مصری عبدل منعم محمود (عربی) پس از آزاد شدنش از زندان تیتر اولین مطلب خود را «من آزاد هستم» (عربی) انتخاب کرد.'], 'category': 'global_voices_en_fa'}


In [6]:
# Randomly sample 20,000 examples from the dataset
sample_size = 60000
sampled_dataset = dataset.shuffle(seed=42).select(range(sample_size))

# Extract English and Persian sentences
english_sentences = [item['source'] for item in sampled_dataset]
persian_sentences = [item['targets'][0] for item in sampled_dataset]

print(f"Number of samples in the subset: {len(english_sentences)}")
print("First English sentence:", english_sentences[0])
print("First Persian sentence:", persian_sentences[0])

Number of samples in the subset: 60000
First English sentence: She ventured to ask: Is it true, Madame?
First Persian sentence: دل بدریا زد پرسید: خانم، این راسته؟


In [7]:
file_path = '/content/drive/MyDrive/DL_HW3/'

In [ ]:
# import pandas as pd
# from datasets import load_dataset

# fa_data = pd.read_csv(file_path + 'persian_corpus.txt', header=None, sep="\n\n", names=['targets'])
# en_data = pd.read_csv(file_path + 'english_corpus.txt', header=None, sep="\n\n", names=['source'])

# fa_df = pd.DataFrame(fa_data, columns=['targets'])
# en_df = pd.DataFrame(en_data, columns=['source'])

# extraced_col = en_df['source']
# data_frame = pd.concat([fa_df, extraced_col.rename('source')], axis=1)

<ipython-input-7-34c1905f51b5>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  fa_data = pd.read_csv(file_path + 'persian_corpus.txt', header=None, sep="\n\n", names=['targets'])


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DL_HW3/persian_corpus.txt'

In [ ]:
# # Randomly sample 20,000 examples from the dataset
# sample_size = 20000
# sampled_dataset = data_frame.sample(frac=1)[:sample_size]

# # Extract English and Persian sentences
# english_sentences = [item['source'] for id, item in sampled_dataset.iterrows()]
# persian_sentences = [item['targets'] for id, item in sampled_dataset.iterrows()]

# print(f"Number of samples in the subset: {len(english_sentences)}")
# print("First English sentence:", english_sentences[0])
# print("First Persian sentence:", persian_sentences[0])

Number of samples in the subset: 20000
First English sentence: you shot me, you stupid bitch!
First Persian sentence: ! تو منو زدي ، توي احمق


## Train valid Test split & Creating Datasets

In [8]:
from sklearn.model_selection import train_test_split

# Define the proportion for the train-test split
test_size = 0.10  # 20% of data for testing
random_seed = 42  # For reproducibility

# Split the data into train and test sets
english_train, english_test, persian_train, persian_test = train_test_split(
    english_sentences, persian_sentences, test_size=test_size, random_state=random_seed
)

english_test, english_valid, persian_test, persian_valid = train_test_split(
    english_test, persian_test, test_size=0.4, random_state=random_seed
)

# Display the size of each split
print(f"Number of training samples: {len(english_train)}")
print(f"Number of validation samples: {len(english_valid)}")
print(f"Number of testing samples: {len(english_test)}")

Number of training samples: 54000
Number of validation samples: 2400
Number of testing samples: 3600


In [9]:
# Custom Dataset class for English-Persian translation
class TranslationDataset(Dataset):
    def __init__(self, src_data, tgt_data):
        self.src_data = src_data
        self.tgt_data = tgt_data

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        return self.src_data[idx], self.tgt_data[idx]

# Create Dataset
train_dataset = TranslationDataset(english_train, persian_train)
valid_dataset = TranslationDataset(english_valid, persian_valid)
test_dataset = TranslationDataset(english_test, persian_test)

## Build vocabulary

In [32]:
token_transform = {}
mb =  MBartTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
def en_tokenize(sentence):
    return mb.tokenize(sentence)
token_transform['en'] = en_tokenize

sp = spm.SentencePieceProcessor()
sp.load(file_path + 'persian_spm.model')
# Define the tokenization function using SentencePiece
def persian_tokenize(sentence):
    return sp.tokenize(sentence, out_type=str)
token_transform['fa'] = persian_tokenize
# تعریف نمادهای ویژه
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

# دیکشنری برای ذخیره واژگان هر زبان
vocab_transform = {}

# تابعی برای ایجاد توکن‌ها از داده‌ها
# Function to yield individual tokens (one by one)
def yield_tokens(data_iter, language: str):
    language_index = {'en': 0, 'fa': 1}
    for i, data_sample in enumerate(data_iter):
        yield token_transform[language](data_sample[language_index[language]])



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


In [33]:
for ln in ['fa', 'en']:
    print(ln)
    # داده‌های زبان را به ترتیب طول جمله‌ها مرتب می‌کنیم
    train_iterator =  list(zip(english_train, persian_train)) # باید مجموعه داده خود را مشخص کنید
    sorted_dataset = sorted(train_iterator, key=lambda x: len(x[0].split()))

    # use build_vocab_from_iterator function and also use special_symbols
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(sorted_dataset, ln), specials=special_symbols)

vocab_transform['en'].set_default_index(UNK_IDX)
vocab_transform['fa'].set_default_index(UNK_IDX)
en_vocab_size = len(vocab_transform['en'])
fa_vocab_size = len(vocab_transform['fa'])
# Print vocab size for verification
print("English vocab size:",en_vocab_size )
print("Persian vocab size:",fa_vocab_size)

print(list(vocab_transform['fa'].get_stoi().keys())[-10:])
print(list(vocab_transform['en'].get_stoi().keys())[:30])

fa
en
English vocab size: 16059
Persian vocab size: 19900
['▁جامع', '▁جانانه', '▁ماکو', '▁جاها', '▁جذبه', '▁درازی', '▁جستند', '▁احدی', '▁کارمند', '▁جلویش']
['▁نگار', '▁zen', '▁yuz', '▁yuk', '▁yin', '▁yay', '▁winst', '▁welches', '▁websites', '▁waka', '▁vu', '▁volt', '▁volatil', '▁voce', '▁visa', '▁violation', '▁vim', '▁vidu', '▁vibrant', '▁versus', '▁vero', '▁verbo', '▁vente', '▁vendor', '▁vender', '▁£', '▁vall', '▁uz', '▁utili', '▁usa']


In [34]:
print(vocab_transform['en'](['I','▁like','▁machine','▁learning']))
sentence1= "من عاشق کتابها هستم"
sentence2= "من"
print(token_transform['fa'](sentence1))
print(vocab_transform['fa'](token_transform['fa'](sentence1)))
print(vocab_transform['fa'](token_transform['fa'](sentence2)))

[584, 72, 3192, 3548]
['▁من', '▁عاشق', '▁کتابها', '▁هستم']
[17, 815, 15076, 250]
[17]


## Prepare data to input model
we make  train_dataloader and  valid_dataloader in this step.

### tensor transforms

tensor_transform_s:
این تابع برای پردازش جملات منبع (مانند انگلیسی در ترجمه انگلیسی به فارسی) برای یک مدل دنباله به دنباله استفاده می‌شود.
    
    1- BOS_IDX:  ‌توکن "آغاز دنباله" را به ابتدای جمله اضافه کنید.


    2- reversing tokens:
     توکن‌های ورودی را معکوس کنید. این معمولاً برای دنباله‌های منبع در برخی از مدل‌های دنباله به دنباله انجام می‌شود، مانند برخی از انواع ترجمه ماشینی عصبی  تا مدل کاراتر شود.

            در برخی از مدل‌های ترجمه ماشینی عصبی ، عملیات برعکس کردن دنباله‌های ورودی (مانند جملات منبع) به‌ویژه برای مدل‌های دنباله به دنباله انجام می‌شود. این کار به دلیل دلایل مختلفی صورت می‌گیرد:

            بهبود یادگیری مدل: با برعکس کردن دنباله‌ها، مدل قادر خواهد بود روابط بلندمدت و وابستگی‌های طولانی‌تر را به‌خوبی یاد بگیرد. این تکنیک به‌ویژه در زمانی که ورودی‌ها وابستگی‌های طولانی دارند، مفید است.

            تسهیل پردازش وابستگی‌ها: در مدل‌های بازگشتی ، توکن‌های انتهایی ممکن است برای پیش‌بینی توکن‌های ابتدایی مفید باشند. برعکس کردن دنباله‌ها این امر را برای مدل راحت‌تر می‌کند تا وابستگی‌های مهم را شناسایی کرده و پردازش کند.

            اهمیت ترتیب توکن‌ها: برعکس کردن دنباله‌ها ممکن است به مدل کمک کند تا ویژگی‌های ساختاری زبان‌های مختلف را به‌خوبی یاد بگیرد، به‌ویژه در مواردی که ترتیب توکن‌ها در زبان‌های مختلف اهمیت دارد.

            این تکنیک به‌طور خاص در مدل‌هایی مانند ترجمه ماشینی استفاده می‌شود تا یادگیری وابستگی‌های متنی بهتر انجام شود و مدل به دقت بیشتری دست یابد.

    
    3- EOS_IDX:  توکن "پایان دنباله" را به انتهای جمله اضافه کنید.

tensor_transform_t:
این تابع برای جملات هدف (مانند فارسی) استفاده می‌شود.

   
    ۱- BOS_IDX: یک توکن "آغاز دنباله" را اضافه می‌کند.

    ۲- بدون برعکس کردن: جملات هدف معمولاً در همان ترتیب خود نگه‌داشته می‌شوند.
   
    ۳- EOS_IDX: یک توکن "پایان دنباله" را به انتهای جمله اضافه می‌کند.

این تبدیل‌ها اطمینان می‌دهند که جملات به درستی توکنایز شده و برای شبکه عصبی ساختاربندی شده‌اند.

تابع sequential_transforms:

این تابع به شما این امکان را می‌دهد که چندین تبدیل را به‌صورت زنجیره‌ای به هم متصل کنید، به طوری که این تبدیل‌ها به ترتیب بر روی متن ورودی اعمال شوند. این تابع چندین تابع تبدیل را می‌گیرد و آن‌ها را به ترتیبی که ارسال شده‌اند، اعمال می‌کند.

In [35]:
# Transform functions for source and target tokens
def tensor_transform_s(token_ids: List[int]):
    return torch.cat((
        torch.tensor([BOS_IDX]),                                        # Add beginning-of-sequence token
        torch.tensor(list(reversed(token_ids)), dtype=torch.int64),     # Reverse the token list (for source sentences)
        torch.tensor([EOS_IDX])                                         # Add end-of-sequence token
    ))

def tensor_transform_t(token_ids: List[int]):
    return torch.cat((
        torch.tensor([BOS_IDX]),                                         # Add beginning-of-sequence token
        torch.tensor(token_ids, dtype=torch.int64),                      # Keep the token list as is
        torch.tensor([EOS_IDX])                                          # Add end-of-sequence token
    ))

def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func



### Define text transformations

In [36]:
# Tokenization and vocabulary for both languages
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'fa'

# Set device to GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
flip = True
text_transform = {}

text_transform[SRC_LANGUAGE] = sequential_transforms(
    token_transform[SRC_LANGUAGE],
    vocab_transform[SRC_LANGUAGE],
    tensor_transform_s if flip else tensor_transform_t
)

text_transform[TGT_LANGUAGE] = sequential_transforms(
    token_transform[TGT_LANGUAGE],
    vocab_transform[TGT_LANGUAGE],
    tensor_transform_s if not flip else tensor_transform_t
)

### collate_fn
  برای آماده‌سازی یک دسته  از داده‌های ورودی برای مدل استفاده می‌شود. این
تابع داده‌های خام را به توکن‌های عددی تبدیل می‌کند، آن‌ها را به طول‌های برابر پر می‌کند ، و سپس آن‌ها را به دستگاهی که مدل روی آن اجرا می‌شود  ارسال می‌کند.

In [37]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

# Collate function to pad and batch the data
def collate_fn(batch):
    #لیست‌های  زیر برای ذخیره توالی‌های ورودی  و خروجی  استفاده می‌شوند.
    src_batch, tgt_batch = [], []
    #هر جفت جمله ورودی  و خروجی  از دسته داده‌ها خوانده می‌شود.
    for src_sample, tgt_sample in batch:

        src_sequences = text_transform[SRC_LANGUAGE](src_sample.rstrip("\n"))
        src_sequences = torch.tensor(src_sequences, dtype=torch.int64)

        tgt_sequences = text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n"))
        tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)

        src_batch.append(src_sequences)
        tgt_batch.append(tgt_sequences)

    seq_lengths = [len(seq) for seq in src_batch]
    src_max_length = max(seq_lengths)

    seq_lengths = [len(seq) for seq in tgt_batch]
    tgt_max_length = max(seq_lengths)

    max_len = max(tgt_max_length, src_max_length)
    #توالی‌های ورودی و خروجی ممکن است طول‌های متفاوتی داشته باشند.  همه توالی‌ها به طول یکسان (طول طولانی‌ترین توالی) پر شوند.
    src_batch = torch.tensor(pad_sequences(src_batch, maxlen=max_len), dtype=torch.int64)
    tgt_batch = torch.tensor(pad_sequences(tgt_batch, maxlen=max_len), dtype=torch.int64)
    # tgt_batch = pad_sequences(tgt_batch, max_len=max_len, padding_value=PAD_IDX)

    #ترتیب ابعاد دسته‌ها از  .
    #[batch_size, seq_len]
    #به
    #[seq_len, batch_size]
    #تغییر دهید. این ترتیب معمولاً برای مدل‌های بازگشتی  مناسب‌تر است
    src_batch = src_batch.transpose(0, 1)
    tgt_batch = tgt_batch.transpose(0, 1)

    return src_batch.to(device), tgt_batch.to(device)


In [38]:
# Function to create DataLoader
def get_translation_dataloaders(batch_size=8):

    # Sort by length of the source sentence for efficiency
    sorted_traindataset = sorted(train_dataset, key=lambda x: len(x[0].split()))
    sorted_validdataset = sorted(valid_dataset, key=lambda x: len(x[0].split()))
    sorted_testdataset = sorted(test_dataset, key=lambda x: len(x[0].split()))

    # Create DataLoader
    train_dataloader = DataLoader(
        sorted_traindataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        drop_last=True
    )

    valid_dataloader = DataLoader(
        sorted_validdataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        drop_last=True
    )
    test_dataloader = DataLoader(
        sorted_testdataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        drop_last=True
    )

    return train_dataloader, valid_dataloader, test_dataloader

## Translation model

In [39]:
train_dataloader, valid_dataloader, test_dataloader = get_translation_dataloaders(batch_size=16)

In [40]:
print(next(iter(train_dataloader))[0].shape)

torch.Size([7, 16])


<ipython-input-37-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-37-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)


In [41]:
print(next(iter(train_dataloader))[1].shape)


torch.Size([7, 16])


<ipython-input-37-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-37-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)


### Load Word2Vec model for persian

In [42]:
word2vec_model = Word2Vec.load(file_path + 'word2vec_persian.model')
vocab_size = fa_vocab_size
embedding_dim = word2vec_model.vector_size

# Initialize embedding matrix
embedding_matrix = np.random.uniform(-0.1, 0.1, (vocab_size, embedding_dim))

# Fill embedding matrix with Word2Vec vectors; fill embedding_matrix with values in word2vec_model
word_to_index = {word: idx for idx, word in enumerate(word2vec_model.wv.index_to_key)}
for word, index in word_to_index.items():
    if word in word2vec_model.wv and index < vocab_size:
        embedding_matrix[index] = word2vec_model.wv[word]

# Convert to torch tensor
embedding_matrix_fa = torch.tensor(embedding_matrix, dtype=torch.float).to(device)
print(embedding_matrix_fa.shape)

torch.Size([19900, 300])


In [43]:
def most_similar_words(word):
    try:
        similar_words = word2vec_model.wv.most_similar(word, topn=10)
        print(f"Most similar words to '{word}':")
        for similar_word, similarity in similar_words:
            print(f"  {similar_word}: {similarity:.4f}")
    except KeyError:
        print(f"'{word}' not in vocabulary!")

# Test with an example word
most_similar_words("دختر")

Most similar words to 'دختر':
  پسر: 0.7491
  زن: 0.6631
  خواهر: 0.6099
  طفل: 0.5996
  مرد: 0.5785
  کودک: 0.5677
  دختری: 0.5567
  همسر: 0.5439
  پیرزن: 0.5435
  برادر: 0.5434


### Load Word2Vec model for english

In [47]:
word2vec_model = Word2Vec.load(file_path + 'word2vec_english.model')
vocab_size = en_vocab_size
embedding_dim = word2vec_model.vector_size

# Initialize embedding matrix
embedding_matrix = np.random.uniform(-0.1, 0.1, (vocab_size, embedding_dim))

# Fill embedding matrix with Word2Vec vectors; fill embedding_matrix with values in word2vec_model
word_to_index = {word: idx for idx, word in enumerate(word2vec_model.wv.index_to_key)}
for word, index in word_to_index.items():
    if word in word2vec_model.wv and index < vocab_size:
        embedding_matrix[index] = word2vec_model.wv[word]

# Convert to torch tensor
embedding_matrix_en = torch.tensor(embedding_matrix, dtype=torch.float).to(device)
print(embedding_matrix_en.shape)

torch.Size([16059, 300])


In [48]:
def most_similar_words(word):
    try:
        similar_words = word2vec_model.wv.most_similar(word, topn=10)
        print(f"Most similar words to '{word}':")
        for similar_word, similarity in similar_words:
            print(f"  {similar_word}: {similarity:.4f}")
    except KeyError:
        print(f"'{word}' not in vocabulary!")

# Test with an example word
most_similar_words("man")

Most similar words to 'man':
  struck: 0.9967
  last: 0.9966
  year: 0.9961
  scarcely: 0.9960
  while: 0.9959
  child: 0.9959
  month: 0.9958
  passed: 0.9956
  woman: 0.9954
  instant: 0.9953


In [72]:
temperature = 1.5

In [73]:
class Encoder(nn.Module):
    def __init__(self, vocab_len, emb_dim, hid_dim, n_layers, dropout_prob):
        super().__init__()
        self.embedding = nn.Embedding(vocab_len, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers)
        self.dropout = nn.Dropout(dropout_prob)
        self.embedding.weight.data.copy_(embedding_matrix_en)
        self.embedding.weight.requires_grad = True

    def forward(self, input_batch):
        embedded = self.dropout(self.embedding(input_batch))
        lstm_output, (hidden, cell) = self.lstm(embedded)
        return hidden, cell

In [74]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers)
        self.linear = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)
        self.embedding.weight.data.copy_(embedding_matrix_fa)
        self.embedding.weight.requires_grad = True
        self.output_dim = output_dim

    def forward(self, input_batch, hidden, cell):
        if input_batch.dim() <= 1:
            input_batch = input_batch.unsqueeze(0)
        embedded = self.dropout(self.embedding(input_batch))
        lstm_out, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        linear_out = self.linear(lstm_out)
        softmax_out = F.softmax(linear_out / temperature, dim=-1)

        return softmax_out, hidden, cell

In [75]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, trg_vocab):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.trg_vocab = trg_vocab

    def forward(self, src, trg, teacher_forcing_ratio=1):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)
        hidden = hidden.to(device)
        cell = cell.to(device)

        input = trg[0, :]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder.forward(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(2).long()
            input = trg[t, :] if teacher_force else top1
        return outputs

In [76]:
emb_dim = 300
hid_dim = 300
n_layers = 5
dropout_prob = 0.2
encoder = Encoder(en_vocab_size, emb_dim, hid_dim, n_layers, dropout_prob)
output_dim = fa_vocab_size
decoder = Decoder(output_dim, emb_dim, hid_dim, n_layers, dropout_prob)
trg_vocab = vocab_transform['fa']
model = Seq2Seq(encoder, decoder, device, trg_vocab).to(device)

## Training and Validation

In [57]:
criterion = nn.NLLLoss(ignore_index=PAD_IDX)

def compute_loss(outputs, trg, trg_vocab_size):
    """
    Compute the loss for the Seq2Seq model.

    Args:
    outputs: Predicted logits from the model, shape (trg_len, batch_size, trg_vocab_size)
    trg: Ground truth target sequence, shape (trg_len, batch_size)
    trg_vocab_size: Size of the target vocabulary

    Returns:
    loss: Computed loss value
    """

    outputs = outputs.view(-1, trg_vocab_size)
    trg = trg.reshape(-1)
    loss = criterion(outputs, trg)
    return loss

In [58]:
def index_to_eng(seq_en):
    return " ".join([vocab_transform['en'].get_itos()[index.item()] for index in seq_en])

def index_to_german(seq_de):
    return " ".join([vocab_transform['de'].get_itos()[index.item()] for index in seq_de])

In [59]:
def train(model, iterator, optimizer, clip):
    model.train()
    epoch_loss = 0
    train_iterator = tqdm(iterator, desc="Training", leave=False)

    for i, (src, trg) in enumerate(train_iterator):

        optimizer.zero_grad()
        output = model.forward(src, trg, teacher_forcing_ratio=0.5)
        loss = compute_loss(output, trg, fa_vocab_size)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        train_iterator.set_postfix(loss=loss.item())
        epoch_loss += loss.item()

    return epoch_loss / len(list(iterator))


In [60]:
def evaluate(model, iterator):
    model.eval()
    epoch_loss = 0
    valid_iterator = tqdm(iterator, desc="Validating", leave=False)

    with torch.no_grad():
        for i, (src, trg) in enumerate(valid_iterator):
            output = model(src, trg, teacher_forcing_ratio=0)
            loss = compute_loss(output, trg, fa_vocab_size)
            valid_iterator.set_postfix(loss=loss.item())
            epoch_loss += loss.item()
    return epoch_loss / len(list(iterator))

In [61]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [62]:
device

device(type='cuda')

In [63]:
def generate_translation(model, src_sentence, src_vocab, trg_vocab, max_len=50):
    model.eval()

    with torch.no_grad():
        src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)
        hidden, cell = model.encoder(src_tensor)
        trg_indexes = [trg_vocab.get_stoi()['<bos>']]                       # Start with <bos> token
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1).to(device)  # Add batch dimension

        hidden = hidden.unsqueeze(1).to(device)    # Add batch dimension
        cell = cell.unsqueeze(1).to(device)        # Add batch dimension

        for i in range(max_len):
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
            pred_token = output[0].argmax(1)
            trg_indexes.append(pred_token)
            if pred_token == trg_vocab.get_stoi()['<eos>']:
                break
            trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1).to(device) # Add batch dimension # fix: move to device
            trg_tokens =  [trg_vocab.get_itos()[i] for i in trg_indexes]

        if trg_tokens[0]=='<bos>':
            trg_tokens = trg_tokens[1:]
        if trg_tokens[-1] == '<eos>':
            trg_tokens = trg_tokens[:-1]

        translation = " ".join(trg_tokens)
        return translation

In [64]:
# Define the optimizer
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_iterator = train_dataloader
valid_iterator = valid_dataloader
clip = 1
tlosses = []
vlosses = []
src_sentences = ["I want to go home.", "you should translate this sentence.", "how are you?", "do you want to eat cake?"]
src_vocab = vocab_transform['en']
trg_vocab = vocab_transform['fa']

In [78]:
epochs = 6

## Using *requires_grad = True*

In [79]:
for i in range(epochs) :
    print(f'\n\t\t\tEpoch {i+1} out of {epochs}')
    train_loss = train(model, train_iterator, optimizer, clip)
    valid_loss = evaluate(model, valid_iterator)
    print(f'Training Loss: {train_loss}')
    print(f'Validation Loss: {valid_loss}')
    tlosses.append(train_loss)
    vlosses.append(valid_loss)
    translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
    print(f'Translation: {translation}')


			Epoch 1 out of 6


Training:   0%|          | 0/3375 [00:00<?, ?it/s]<ipython-input-37-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-37-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)


Training Loss: -4.784271549929313e-05
Validation Loss: -4.79348447212639e-05


<ipython-input-63-9a426f65e639>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Translation: اگر ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه

			Epoch 2 out of 6


Training Loss: -4.7842715368864854e-05
Validation Loss: -4.79348447212639e-05
Translation: اگر ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه

			Epoch 3 out of 6


Training Loss: -4.784273394573204e-05
Validation Loss: -4.79348447212639e-05
Translation: اگر ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه

			Epoch 4 out of 6


Training Loss: -4.784272426816945e-05
Validation Loss: -4.79348447212639e-05
Translation: اگر ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه

			Epoch 5 out of 6


Training Loss: -4.784276678239823e-05
Validation Loss: -4.79348447212639e-05
Translation: اگر ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه

			Epoch 6 out of 6


Training Loss: -4.7842716943705453e-05
Validation Loss: -4.79348447212639e-05
Translation: اگر ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁چونکه ▁نعل ▁نعل ▁نعل ▁نعل ▁نعل ▁چونکه ▁چونکه ▁چونکه ▁چونکه


In [71]:
for i in range(epochs) :
    print(f'\n\t\t\tEpoch {i+1} out of {epochs}')
    train_loss = train(model, train_iterator, optimizer, clip)
    valid_loss = evaluate(model, valid_iterator)
    print(f'Training Loss: {train_loss}')
    print(f'Validation Loss: {valid_loss}')
    tlosses.append(train_loss)
    vlosses.append(valid_loss)
    translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
    print(f'Translation: {translation}')


			Epoch 1 out of 5


Training:   0%|          | 0/3375 [00:00<?, ?it/s]<ipython-input-37-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-37-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)


Training Loss: -4.799778450763336e-05
Validation Loss: -4.811375561985187e-05


<ipython-input-63-9a426f65e639>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Translation: ▁متعاقب ▁متعاقب ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف

			Epoch 2 out of 5


Training Loss: -4.799776756489236e-05
Validation Loss: -4.811375561985187e-05
Translation: ▁متعاقب ▁متعاقب ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف ▁وسف

			Epoch 3 out of 5


KeyboardInterrupt: 

# *Using Temerature = 1.3*

In [ ]:
for i in range(epochs) :
    print(f'\n\t\t\tEpoch {i+1} out of {epochs}')
    train_loss = train(model, train_iterator, optimizer, clip)
    valid_loss = evaluate(model, valid_iterator)
    print(f'Training Loss: {train_loss}')
    print(f'Validation Loss: {valid_loss}')
    tlosses.append(train_loss)
    vlosses.append(valid_loss)
    translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
    print(f'Translation: {translation}')


			Epoch 1 out of 2


Training:   0%|          | 0/1687 [00:00<?, ?it/s]<ipython-input-187-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-187-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)
Training:  81%|████████  | 1365/1687 [02:53<01:07,  4.79it/s, loss=-0.315]

In [ ]:
for i in range(epochs) :
    print(f'\n\t\t\tEpoch {i+1} out of {epochs}')
    train_loss = train(model, train_iterator, optimizer, clip)
    valid_loss = evaluate(model, valid_iterator)
    print(f'Training Loss: {train_loss}')
    print(f'Validation Loss: {valid_loss}')
    tlosses.append(train_loss)
    vlosses.append(valid_loss)
    translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
    print(f'Translation: {translation}')


			Epoch 1 out of 3


Training:   0%|          | 0/675 [00:00<?, ?it/s]<ipython-input-69-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-69-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)
<ipython-input-35-9510e3931573>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Training Loss: -0.3130190773510279
Validation Loss: -0.3786133070786794
Translation: <unk> <unk> <unk>

			Epoch 2 out of 3


Training Loss: -0.3256018424254877
Validation Loss: -0.3772079885005951
Translation: <unk> <unk> <unk>

			Epoch 3 out of 3


Training Loss: -0.32580314256526804
Validation Loss: -0.3736504962046941
Translation: <unk> <unk>


In [ ]:
for i in range(epochs) :
    print(f'\n\t\t\tEpoch {i+4} out of {epochs + 4}')
    train_loss = train(model, train_iterator, optimizer, clip)
    valid_loss = evaluate(model, valid_iterator)
    print(f'Training Loss: {train_loss}')
    print(f'Validation Loss: {valid_loss}')
    tlosses.append(train_loss)
    vlosses.append(valid_loss)
    translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
    print(f'Translation: {translation}')


			Epoch 4 out of 14


Training:   0%|          | 0/675 [00:00<?, ?it/s]<ipython-input-69-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-69-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)
<ipython-input-35-9510e3931573>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Training Loss: -0.3257935870797546
Validation Loss: -0.3775422915816307
Translation: <unk> <unk> <unk>

			Epoch 5 out of 14


Training Loss: -0.3258156705564923
Validation Loss: -0.37611539314190545
Translation: <unk> <unk> <unk>

			Epoch 6 out of 14


Training Loss: -0.32581741127702923
Validation Loss: -0.37615223079919813
Translation: <unk> <unk>

			Epoch 7 out of 14


...

Training Loss: -0.3258196441553257
Validation Loss: -0.3760841801762581
Translation: <unk> <unk> <unk>

			Epoch 10 out of 14


Training Loss: -0.3258198992411296
Validation Loss: -0.375989523033301
Translation: <unk> <unk>

			Epoch 11 out of 14


Training Loss: -0.32582007233743315
Validation Loss: -0.37619756658871967
Translation: <unk> <unk> <unk>

			Epoch 12 out of 14


Training Loss: -0.32582023472697647
Validation Loss: -0.37619126538435615
Translation: <unk> <unk> <unk>

			Epoch 13 out of 14


Training Loss: -0.3258203215952273
Validation Loss: -0.3762066940466563
Translation: <unk> <unk> <unk>


# *Using Temerature = 0.5*

In [ ]:
for i in range(epochs) :
    print(f'\n\t\t\tEpoch {i+1} out of {epochs}')
    train_loss = train(model, train_iterator, optimizer, clip)
    valid_loss = evaluate(model, valid_iterator)
    print(f'Training Loss: {train_loss}')
    print(f'Validation Loss: {valid_loss}')
    tlosses.append(train_loss)
    vlosses.append(valid_loss)
    translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
    print(f'Translation: {translation}')


			Epoch 1 out of 2


Training:   0%|          | 0/1125 [00:00<?, ?it/s]<ipython-input-121-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-121-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)
<ipython-input-35-9510e3931573>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Training Loss: -0.36834552318773545
Validation Loss: -0.38355612456798555
Translation: <unk> <unk> <unk>

			Epoch 2 out of 2


Training Loss: -0.37523928291267816
Validation Loss: -0.3846373191475868
Translation: <unk> <unk> <unk>


In [ ]:
for i in range(epochs) :
    print(f'\n\t\t\tEpoch {i} out of {epochs}')
    train_loss = train(model, train_iterator, optimizer, clip)
    valid_loss = evaluate(model, valid_iterator)
    print(f'Training Loss: {train_loss}')
    print(f'Validation Loss: {valid_loss}')
    tlosses.append(train_loss)
    vlosses.append(valid_loss)
    translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
    print(f'Translation: {translation}')


			Epoch 1 out of 2


Training:   0%|          | 0/6750 [00:00<?, ?it/s]<ipython-input-18-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-18-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)
<ipython-input-35-9510e3931573>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Training Loss: -0.3262248799676432
Validation Loss: -0.33577304328481355
Translation: <unk> <unk> <unk>

			Epoch 2 out of 2


Training Loss: -0.3434584260218673
Validation Loss: -0.31771285166343055
Translation: <unk> ▁. ▁. ▁.


In [ ]:
for i in range(epochs) :
    print(f'\n\t\t\tEpoch {i+3} out of {epochs+2}')
    train_loss = train(model, train_iterator, optimizer, clip)
    valid_loss = evaluate(model, valid_iterator)
    print(f'Training Loss: {train_loss}')
    print(f'Validation Loss: {valid_loss}')
    tlosses.append(train_loss)
    vlosses.append(valid_loss)
    translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
    print(f'Translation: {translation}')


			Epoch 3 out of 7


Training:   0%|          | 0/675 [00:00<?, ?it/s]<ipython-input-69-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-69-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)
<ipython-input-35-9510e3931573>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Training Loss: -0.3185578900882629
Validation Loss: -0.37648581465085346
Translation: <unk> <unk> <unk>

			Epoch 4 out of 7


Training Loss: -0.3256458223969848
Validation Loss: -0.3778688500324885
Translation: <unk> <unk> <unk>

			Epoch 5 out of 7


Training Loss: -0.3344394911218573
Validation Loss: -0.36394492437442144
Translation: ▁. ▁. ▁.

			Epoch 6 out of 7


Training Loss: -0.3255698111763707
Validation Loss: -0.3830207705497742
Translation: <unk> <unk> <unk> <unk>

			Epoch 7 out of 7


Training Loss: -0.3258203442229165
Validation Loss: -0.3829743847250938
Translation: <unk> <unk> <unk>


# *Using Temerature = 1*

In [ ]:
for i in range(epochs) :
  print(f'\n\t\t\tEpoch {i+1} out of {epochs}')
  train_loss = train(model, train_iterator, optimizer, clip)
  valid_loss = evaluate(model, valid_iterator)
  print(f'Training Loss: {train_loss}')
  print(f'Validation Loss: {valid_loss}')
  tlosses.append(train_loss)
  vlosses.append(valid_loss)
  translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
  print(f'Translation: {translation}')


			Epoch 1 out of 2


Training:   0%|          | 0/1125 [00:00<?, ?it/s]<ipython-input-121-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-121-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)


Training Loss: -0.31565967519321
Validation Loss: -0.33139513462781905


<ipython-input-35-9510e3931573>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>

			Epoch 2 out of 2


Training Loss: -0.3208350900742743
Validation Loss: -0.33139812290668486
Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>


In [ ]:
for i in range(epochs) :
  print(f'\n\t\t\tEpoch {i+1} out of {epochs}')
  train_loss = train(model, train_iterator, optimizer, clip)
  valid_loss = evaluate(model, valid_iterator)
  print(f'Training Loss: {train_loss}')
  print(f'Validation Loss: {valid_loss}')
  tlosses.append(train_loss)
  vlosses.append(valid_loss)
  translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
  print(f'Translation: {translation}')


			Epoch 1 out of 5


Training:   0%|          | 0/1125 [00:00<?, ?it/s]<ipython-input-121-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-121-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)


Training Loss: -0.3198357956667088
Validation Loss: -0.33139941334724426


<ipython-input-35-9510e3931573>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>

			Epoch 2 out of 5


Training Loss: -0.32083749451902177
Validation Loss: -0.33139941334724426
Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>

			Epoch 3 out of 5


Training Loss: -0.32083749451902177
Validation Loss: -0.33139941334724426
Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>

			Epoch 4 out of 5


Training Loss: -0.32083749451902177
Validation Loss: -0.33139941334724426
Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>

			Epoch 5 out of 5


Training Loss: -0.32083749451902177
Validation Loss: -0.33139941334724426
Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>


In [ ]:
for i in range(epochs) :
  print(f'\n\t\t\tEpoch {i+1} out of {epochs}')
  train_loss = train(model, train_iterator, optimizer, clip)
  valid_loss = evaluate(model, valid_iterator)
  print(f'Training Loss: {train_loss}')
  print(f'Validation Loss: {valid_loss}')
  tlosses.append(train_loss)
  vlosses.append(valid_loss)
  translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
  print(f'Translation: {translation}')


			Epoch 1 out of 5


Training:   0%|          | 0/1125 [00:00<?, ?it/s]<ipython-input-121-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-121-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)


Training Loss: -0.32083749451902177
Validation Loss: -0.33139941334724426


<ipython-input-35-9510e3931573>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>

			Epoch 2 out of 5


Training Loss: -0.32083749451902177
Validation Loss: -0.33139941334724426
Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>

			Epoch 3 out of 5


Training Loss: -0.32083749451902177
Validation Loss: -0.33139941334724426
Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>

			Epoch 4 out of 5


Training Loss: -0.32083749451902177
Validation Loss: -0.33139941334724426
Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>

			Epoch 5 out of 5


Training Loss: -0.32083749451902177
Validation Loss: -0.33139941334724426
Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>


In [ ]:
for i in range(epochs) :
  print(f'\n\t\t\tEpoch {i} out of {epochs}')
  train_loss = train(model, train_iterator, optimizer, clip)
  valid_loss = evaluate(model, valid_iterator)
  print(f'Training Loss: {train_loss}')
  print(f'Validation Loss: {valid_loss}')
  tlosses.append(train_loss)
  vlosses.append(valid_loss)
  translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
  print(f'Translation: {translation}')


			Epoch 0 out of 5


Training:   0%|          | 0/1125 [00:00<?, ?it/s]<ipython-input-121-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-121-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)
<ipython-input-35-9510e3931573>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Training Loss: -0.36565243700905314
Validation Loss: -0.37987531304359434
Translation: <unk>

			Epoch 1 out of 5


Training Loss: -0.37095387577679423
Validation Loss: -0.38205392777919767
Translation: <unk> <unk> <unk> <unk>

			Epoch 2 out of 5


Training Loss: -0.36595066946393895
Validation Loss: -0.379331833422184
Translation: <unk> <unk> <unk>

			Epoch 3 out of 5


Training Loss: -0.373570165236791
Validation Loss: -0.38436281502246855
Translation: <unk> <unk> <unk>

			Epoch 4 out of 5


Training Loss: -0.3754229121208191
Validation Loss: -0.38429475963115695
Translation: <unk> <unk> <unk>


In [ ]:
for i in range(epochs) :
  print(f'\n\t\t\tEpoch {i} out of {epochs}')
  train_loss = train(model, train_iterator, optimizer, clip)
  valid_loss = evaluate(model, valid_iterator)
  print(f'Training Loss: {train_loss}')
  print(f'Validation Loss: {valid_loss}')
  tlosses.append(train_loss)
  vlosses.append(valid_loss)
  translation = generate_translation(model, src_sentences[i % 4], src_vocab, trg_vocab)
  print(f'Translation: {translation}')


			Epoch 0 out of 7


Training:   0%|          | 0/675 [00:00<?, ?it/s]<ipython-input-69-95f95f2b2e0d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
<ipython-input-69-95f95f2b2e0d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)
<ipython-input-35-9510e3931573>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(text_transform['en'](src_sentence), dtype=torch.int64).to(device)


Training Loss: -0.31438962130116394
Validation Loss: -0.37317368884881336
Translation: <unk> <unk> <unk>

			Epoch 1 out of 7


Training Loss: -0.32579305531802
Validation Loss: -0.373583743472894
Translation: <unk> <unk> <unk>

			Epoch 2 out of 7


Training Loss: -0.3258123289434998
Validation Loss: -0.37700083255767824
Translation: <unk> <unk>

			Epoch 3 out of 7


Training Loss: -0.32581780398333515
Validation Loss: -0.3756124387184779
Translation: <unk> <unk> <unk>

			Epoch 4 out of 7


Training Loss: -0.32581834896847056
Validation Loss: -0.37835146536429726
Translation: <unk> <unk> <unk>

			Epoch 5 out of 7


Training Loss: -0.3375222533720511
Validation Loss: -0.38971659590800606
Translation: <unk> <unk> <unk> ▁.

			Epoch 6 out of 7


Training Loss: -0.3427418444112495
Validation Loss: -0.37734451989332835
Translation: ▁. ▁. ▁.


In [ ]:
import matplotlib.pyplot as plt

plt.plot(epochs, tlosses, label='Training Loss')
plt.plot(epochs, vlosses, label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()